In [ ]:
%pip install spacy
%pip install ipynb



In [8]:
%pip install nbimporter


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# %run python -m spacy download en_core_web_sm


In [12]:
import xgboost
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import librosa
import numpy as np


import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv(r'..\data\Data\features_3_sec.csv')

# df = df.drop(['filename'], axis=1)

# df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','label']]

df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


df['label'] =  label_encoder.fit_transform(df['label'])

print(label_encoder.classes_)

y = df[['label']]
X = df[df.columns.difference(['label'])]


## split both X and y using a ratio of 70% training - 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(len(X_train), len(X_test), len(y_train), len(y_test))

xgb = xgboost.XGBClassifier(n_estimators=1000,enable_catergorical=True,learning_rate=0.05)
xgb.fit(X_train, y_train)

## make predictions on the test portion (predict the labels of the rows from the test portion of X)
predictions = xgb.predict(X_test)

target_name = ['blues', 'classical', 'country', 'disco', 'hiphop' ,'jazz' ,'metal', 'pop','reggae' ,'rock']


print(classification_report(y_test, predictions, target_names=target_name))
## can also output the confusion matrix
# cm = confusion_matrix(y_test, predictions)
# print(cm)

print("Accuracy: " ,metrics.accuracy_score(y_test, predictions))

cols_when_model_builds = xgb.feature_names_in_


['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']
6993 2997 6993 2997
              precision    recall  f1-score   support

       blues       0.77      0.83      0.80       302
   classical       0.91      0.92      0.92       296
     country       0.68      0.76      0.72       312
       disco       0.78      0.78      0.78       283
      hiphop       0.85      0.79      0.82       320
        jazz       0.80      0.81      0.80       289
       metal       0.87      0.89      0.88       303
         pop       0.84      0.77      0.80       298
      reggae       0.78      0.73      0.75       310
        rock       0.65      0.61      0.63       284

    accuracy                           0.79      2997
   macro avg       0.79      0.79      0.79      2997
weighted avg       0.79      0.79      0.79      2997

Accuracy:  0.7911244577911245


In [22]:
def extract_features(file):
    y, sr = librosa.load(file)

    chroma_sft_mean =  np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    chroma_sft_var =  librosa.feature.chroma_stft(y=y, sr=sr).var()

    rms_mean = librosa.feature.rms(y=y).mean()
    rms_var = librosa.feature.rms(y=y).var()

    spectral_centroid_mean = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    spectral_centroid_var = librosa.feature.spectral_centroid(y=y, sr=sr).var()

    spectral_bandwith_mean = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    spectral_bandwith_var = librosa.feature.spectral_bandwidth(y=y, sr=sr).var()

    rolloff_mean = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()
    rolloff_var = librosa.feature.spectral_rolloff(y=y, sr=sr).var()

    zero_crossing_rate_mean = librosa.feature.zero_crossing_rate(y=y).mean()
    zero_crossing_rate_var = librosa.feature.zero_crossing_rate(y=y).var()


    harmony_mean = librosa.effects.harmonic(y).mean()
    harmony_var = librosa.effects.harmonic(y).var()

    tempo = librosa.feature.tempo(y=y, sr=sr)[0]

    

    print("chroma_sft_mean: ",chroma_sft_mean)
    print("chroma_sft_var: ",chroma_sft_var)
    print("rms_mean: ",rms_mean)
    print("rms_var: ",rms_var)
    print("spectral_centroid_mean: ",spectral_centroid_mean)
    print("spectral_centroid_var: ",spectral_centroid_var)
    print("spectral_bandwith_mean: ",spectral_bandwith_mean)
    print("spectral_bandwith_var: ",spectral_bandwith_var)
    print("rolloff_mean: ",rolloff_mean)
    print("rolloff_var: ",rolloff_var)
    print("zero_crossing_rate_mean: ",zero_crossing_rate_mean)
    print("zero_crossing_rate_var: ",zero_crossing_rate_var)
    print("harmony_mean: ",harmony_mean)
    print("harmony_var: ",harmony_var)
    print("tempo: ",tempo)

    features = pd.DataFrame({'chroma_stft_mean':[chroma_sft_mean],'chroma_stft_var':[chroma_sft_var],'rms_mean':[rms_mean],'rms_var':[rms_var],'spectral_centroid_mean':[spectral_centroid_mean],
                             'spectral_centroid_var':[spectral_centroid_var],'spectral_bandwidth_mean':[spectral_bandwith_mean],'spectral_bandwidth_var':[spectral_bandwith_var],
                             'rolloff_mean':[rolloff_mean],'rolloff_var':[rolloff_var],'zero_crossing_rate_mean':[zero_crossing_rate_mean],'zero_crossing_rate_var':[zero_crossing_rate_var],
                             'harmony_mean':[harmony_mean],'harmony_var':[harmony_var],'tempo':[tempo],})
    

    features = features.reindex(columns=cols_when_model_builds)

    # features = features.reshape(-1,1)
    return features



In [23]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

def find_sim(data):
    placeHoldername = 'test'
    data['filename'] = placeHoldername

    df_sim = pd.read_csv(r'..\data\Data\features_30_sec.csv')

    df_sim = df_sim[['filename','chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]


    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    df_sim['label'] = df_sim['label'].astype("string")
    df_sim['label'] =  label_encoder.fit_transform(df_sim['label'])




    combined_df = pd.concat([df_sim, data], ignore_index=True)

    combined_df = combined_df.set_index('filename')

 
    labels = combined_df[['label']]

    
    scaled = preprocessing.scale(combined_df)
    similarity = cosine_similarity(scaled)
    sim_df_labels = pd.DataFrame(similarity)
    sim_df_names = sim_df_labels.set_index(labels.index)
    sim_df_names.columns = labels.index

    series = sim_df_names[placeHoldername].sort_values(ascending=False)
    series = series.drop(placeHoldername)
    return series.head(3)





In [24]:
def find_pred(data, features, predicted_feature):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression

    features = features.drop(['filename'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(data, data[predicted_feature], test_size=0.2)

    model = LinearRegression()
    model.fit(X_train, y_train)


    prediction = model.predict(features)

    return prediction[0]

In [62]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

patterns = [
    [{"LOWER": "hello"}],
    [{"LOWER": "hi"}],
    [{"LOWER": "how"}, {"LOWER": "are"}, {"LOWER": "you"}],
    # [{"LOWER": "find"}, {"LOWER": "this"}, {"LOWER": "song"}, {"LOWER": "but"}, {"LOWER": {"REGEX": ".*"}}],
]

responses = {
    "greetings": [
        [{"LOWER": "hello"}],
        [{"LOWER": "hi"}],
    ],
    "inquiries": [
        [{"LOWER": "how"}, {"LOWER": "are"}, {"LOWER": "you"}],
    ],
    "commands": [
        [{"LOWER": "help"}],
    ],

    "find_increased": [
        [{"LOWER": "find"}],
    ],
}


fast_words = ["faster","quicker"]
slow_words = ["slower","calmer"]


for category, patterns in responses.items():
    for pattern in patterns:
        matcher.add(category, [pattern])




In [66]:
def chatbot_response(user_input, features1):
   doc = nlp(user_input)
   matches = matcher(doc)
   if matches:
       match_id, start,end = matches[0]
       category = nlp.vocab.strings[match_id]
       if category == "greetings":
           return "Hello! How can I assist you?"
       elif category == "inquiries":
           return "I'm just a chatbot. How can I assist you?"
       elif category == "commands":
           return "You can ask me for help."
       
       elif category == "find_increased":
            words = [token.text for token in doc if token.is_alpha]
            valid = False
            for s in words:
                if(s in fast_words):
                    features = 'tempo'
                    value = 400
                    valid=True
                elif(s in slow_words):
                    features = 'tempo'
                    value = -400
                    valid=True

            if(valid):
                new_features = features1
                new_features[features] = value
                return find_sim(new_features)
            else:
                return "I'm sorry, but im going to need a valid song feature"

   else:
       return "I'm sorry, I don't understand that."


In [67]:

while True:
    user_input = input("You: ")
    if user_input.lower() == "extract":
        print("Loading....")  
        features1 = extract_features(r"..\music\Beyoncé-Single Ladies.mp3")
        print("Extracted")
        genre1 = xgb.predict(features1)
        print(genre1)
        features1['label'] = genre1[0]

    elif user_input.lower() == "sim":
        print("Similiar Songs:",find_sim(features1))
    elif user_input.lower() == "exit":
        break
    else:
        response = chatbot_response(user_input,features1)
        print(f"Orpheus: {response}")


Orpheus: Hello! How can I assist you?
Orpheus: filename
hiphop.00015.wav    0.869704
rock.00022.wav      0.804720
jazz.00077.wav      0.799673
Name: test, dtype: float64
Orpheus: filename
metal.00066.wav    0.659675
metal.00045.wav    0.659675
rock.00023.wav     0.601153
Name: test, dtype: float64
Orpheus: filename
metal.00066.wav    0.659675
metal.00045.wav    0.659675
rock.00023.wav     0.601153
Name: test, dtype: float64
Orpheus: filename
metal.00066.wav    0.659675
metal.00045.wav    0.659675
rock.00023.wav     0.601153
Name: test, dtype: float64
Orpheus: I'm sorry, but im going to need a valid song feature
